# Day 2022-15: Beacon Exclusion Zone

In [ ]:
year = 2022
day  = 15

In [ ]:
from local_settings import load_input
content = load_input(year, day)
print(f"[{content[:100]}...]")

# Part 1

In [ ]:
# definitions for first part of problem solution
import re

linePattern = re.compile(r"Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)")

def parseInp(s):
    sensors = list()
    beacons = set()
    for line in s.splitlines():
        m = linePattern.match(line)
        if m:
            sx, sy, bx, by = map(int, m.groups())
            dist = abs(bx - sx) + abs(by - sy)
            sensors.append((complex(sx, sy), dist))
            beacons.add(complex(bx, by))
        else:
            print(line)
    return sensors, beacons

def rangesInLine(sensors, linenr):
    for sensor in sensors:
        ydist = abs(sensor[0].imag - linenr)
        if ydist > sensor[1]:
            continue
        ydist = sensor[1] - ydist
        yield (sensor[0].real - ydist, sensor[0].real + ydist + 1)

def mergeRanges(ranges):
    ranges = sorted(ranges)
    l = list()
    lastRange = ranges[0]
    for range in ranges[1:]:
        if range[0] > lastRange[1] + 1:
            l.append(lastRange)
            lastRange = range
        else:
            lastRange = (min(range[0], lastRange[0]), max(range[1], lastRange[1]))
    l.append(lastRange)
    return l

def countNonBeacon(ranges, beacons, line):
    count = 0
    for range in ranges:
        count += range[1] - range[0] - 1
        countBeacons = sum((range[0] <= beacon.real < range[1] for beacon in beacons if beacon.imag == line))
        count -= countBeacons
    return int(count)

## Examples:
```
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
```

In [ ]:
# testing the examples
ex = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""
sensors, beacons = parseInp(ex)
print("Ex 1.1:", countNonBeacon(mergeRanges(rangesInLine(sensors, 10)), beacons, 10))

In [ ]:
# finding the solution
sensors, beacons = parseInp(content)
print("Part 1:", countNonBeacon(mergeRanges(rangesInLine(sensors, 2000000)), beacons, 2000000))

# Part 2

In [ ]:
# definitions for second part of a problem solution
def findCommonSegment(sens1, sens2):
    if sens1[0].real > sens2[0].real:
        sens1, sens2 = sens2, sens1
    pos1, d1 = sens1
    pos2, d2 = sens2
    dist = pos1 - pos2
    dist = int(abs(dist.real)) + int(abs(dist.imag))
    sumDist = abs(d1) + abs(d2)
    if sumDist + 2 == dist:
        x1 = int(pos1.real)
        y1 = int(pos1.imag)
        x2 = int(pos2.real)
        y2 = int(pos2.imag)
        xx1 = max(x1, x2 - d2 - 1)
        xx2 = min(x2, x1 + d1 + 1)
        if y2 > y1:
            yy1 = min(y2, y1 + d1 + 1)
            yy2 = max(y1, y2 - d2 - 1)
            delta = complex(1, -1)
        else:
            yy1 = max(y2, y1 - d1 - 1)
            yy2 = min(y1, y2 + d2 + 1)
            delta = complex(1, 1)
        return complex(xx1, yy1), complex(xx2, yy2), delta
    else:
        return None

def findAllCommonSegments(sensors):
    for sens1idx in range(len(sensors)):
        sens1 = sensors[sens1idx]
        for sens2idx in range(sens1idx + 1, len(sensors)):
            sens2 = sensors[sens2idx]
            common = findCommonSegment(sens1, sens2)
            if common:
                yield common

def findAllCommonPoints(sensors):
    for segm1 in findAllCommonSegments(sensors):
        if segm1[-1] != complex(1, 1):
            continue
        for segm2 in findAllCommonSegments(sensors):
            if segm2[-1] != complex(1, -1):
                continue
            segments = sorted((segm1, segm2), key=lambda x:(x[0].real, x[0].imag))
            p1, p2 = segments[0][0], segments[1][0]
            delta = p2.real - p1.real
            p1 = p1 + delta * segments[0][-1]
            if int(abs(p1.imag - p2.imag)) % 2 != 0:
                continue
            delta = int(abs(p1.imag - p2.imag)) // 2
            p1 = p1 + delta * segments[0][-1]
            p2 = p2 + delta * segments[1][-1]
            if p1 == p2:
                yield p1

def testPoint(point, sensors, beacons):
    if point in beacons:
        return False
    for sensor in sensors:
        dist = point - sensor[0]
        if abs(dist.real) + abs(dist.imag) <= sensor[1]:
            return False
    return True

## Examples:
```
```

In [ ]:
# testing the examples
sensors, beacons = parseInp(ex)
sensors.sort(key=lambda z:(z[0].real, z[0].imag))
solutions = {point for point in findAllCommonPoints(sensors) if testPoint(point, sensors, beacons)}
solution = solutions.pop()
print("Ex 2.1:", int(solution.real * 4_000_000 + solution.imag))

In [ ]:
# finding the solution
sensors, beacons = parseInp(content)
sensors.sort(key=lambda z:(z[0].real, z[0].imag))
solutions = {point for point in findAllCommonPoints(sensors) if testPoint(point, sensors, beacons)}
solution = solutions.pop()
print("Part 2:", int(solution.real * 4_000_000 + solution.imag))